<a href="https://www.kaggle.com/code/sharanharsoor/text-generation-with-an-rnn?scriptVersionId=136683369" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<div class="alert alert-block alert-info">
<h2>Introduction :  Text generation with an RNN </h2> <br/> 
In the realm of text generation, following the advent of ChatGPT, it becomes crucial to grasp the utilization of Deep Learning architecture for generating textual content. Within this notebook, we shall delve into the tutorial on employing RNN (Recurrent Neural Network) for text generation provided by Google. The majority of the code presented herein is sourced directly from the aforementioned text generation tutorial. <br/>

Index:

<ol>
    <li><h4><a href="#Setup">Setup</a></h4></li>
    <li><h4><a href="#Process">Process the text</a></h4></li>
    <li><h4><a href="#Build">Build The Model</a></h4></li>
    <li><h4><a href="#Model">Try the model</a></h4></li>
    <li><h4><a href="#Train">Train the model</a></h4></li>
    <li><h4><a href="#Generate">Generate text</a></h4></li>    
    <li><h4><a href="#Export">Export the generator</a></h4></li>
    <li><h4><a href="#Advance">Advanced: Customized Training</a></h4></li>
</ol>
</div>

### link to [Google](https://www.tensorflow.org/text/tutorials/text_generation) tutorial.


<div class="alert alert-block alert-warning">
<h2> 1. Setup </h2> 
<div id="Setup"></div>
</div>

## Import TensorFlow and other libraries

In [1]:
import tensorflow as tf

import numpy as np
import os
import time

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

## Download the Shakespeare dataset.
Change the following line to run this code on your own data.


In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


## Read the data
First, look in the text:

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


<div class="alert alert-block alert-warning">
<h2> 2. Process the text </h2> 
<div id="Process"></div>
</div>

Vectorize the text

Before training, you need to convert the strings to a numerical representation. 

The `tf.keras.layers.StringLookup` layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [6]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Now create the `tf.keras.layers.StringLookup` layer:

In [7]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

It converts from tokens to character IDs:

In [8]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this you can use `tf.keras.layers.StringLookup(..., invert=True)`.  

Note: Here instead of passing the original vocabulary generated with `sorted(set(text))` use the `get_vocabulary()` method of the `tf.keras.layers.StringLookup` layer so that the `[UNK]` tokens is set the same way.

In [9]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

This layer recovers the characters from the vectors of IDs, and returns them as a `tf.RaggedTensor` of characters:

In [10]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

You can `tf.strings.reduce_join` to join the characters back into strings. 

In [11]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [12]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [13]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [14]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [15]:
seq_length = 100


The `batch` method lets you easily convert these individual characters to sequences of the desired size.

In [16]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


It's easier to see what this is doing if you join the tokens back into strings:

In [17]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


For training you'll need a dataset of `(input, label)` pairs. Where `input` and 
`label` are sequences. At each time step the input is the current character and the label is the next character. 

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [18]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [19]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [20]:
dataset = sequences.map(split_input_target)

In [21]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


### Create training batches

You used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [22]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

<div class="alert alert-block alert-warning">
<h2> 3. Build The Model </h2> 
<div id="Build"></div>
</div>

This section defines the model as a `keras.Model` subclass (For details see [Making new Layers and Models via subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models)). 

This model has three layers:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map each character-ID to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use an LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [23]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [24]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [25]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](https://www.tensorflow.org/static/text/tutorials/images/text_generation_training.png)

Note: For training you could use a `keras.Sequential` model here. To  generate text later you'll need to manage the RNN's internal state. It's simpler to include the state input and output options upfront, than it is to rearrange the model architecture later. For more details see the [Keras RNN guide](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse).

<div class="alert alert-block alert-warning">
<h2> 4. Try the model </h2> 
<div id="Model"></div>
</div>

Now run the model to see that it behaves as expected.

First check the shape of the output

In [26]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [27]:
model.summary()


Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4,022,850
Trainable params: 4,022,850
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [28]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [29]:
sampled_indices

array([50, 20, 43, 16, 65,  9,  0, 23,  3, 49, 40, 61, 11,  4, 27,  8, 13,
       43,  4, 13, 43, 17, 10, 50, 17, 48, 33, 42, 36, 40, 18, 27, 24,  5,
        1,  2, 56, 35, 55, 61, 12,  0, 57, 32, 32, 26, 59, 65, 55, 29, 31,
       24,  5, 18, 26, 35, 44, 48,  8,  7, 35, 60, 60, 53, 27,  5, 48, 48,
       37, 62, 62, 37, 21, 60, 58, 45, 43,  1,  9, 55, 56, 52, 18, 41, 30,
       12, 48, 56, 10,  2, 20, 59,  8, 20, 50, 21, 32,  2,  9, 39])

Decode these to see the text predicted by this untrained model:

In [30]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b"here,\nknock me well, and knock me soundly'? And come you\nnow with, 'knocking at the gate'?\n\nPETRUCHI"

Next Char Predictions:
 b'kGdCz.[UNK]J!jav:$N-?d$?dD3kDiTcWaENK&\n qVpv;[UNK]rSSMtzpPRK&EMVei-,VuunN&iiXwwXHusfd\n.pqmEbQ;iq3 Gt-GkHS .Z'


<div class="alert alert-block alert-warning">
<h2> 5. Train the model </h2> 
<div id="Generate"></div>
</div>

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function
The standard tf.keras.losses.sparse_categorical_crossentropy loss function works in this case because it is applied across the last dimension of the predictions.

Because your model returns logits, you need to set the from_logits flag.




In [31]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [32]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.1892757, shape=(), dtype=float32)


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [33]:
tf.exp(example_batch_mean_loss).numpy()

65.97499

Configure the training procedure using the tf.keras.Model.compile method. Use tf.keras.optimizers.Adam with default arguments and the loss function.

In [34]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [35]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training
To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [36]:
EPOCHS = 20

In [37]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 13s 44ms/step - loss: 2.7181
Epoch 2/20
172/172 [==============================] - 9s 38ms/step - loss: 1.9910
Epoch 3/20
172/172 [==============================] - 9s 39ms/step - loss: 1.7114
Epoch 4/20
172/172 [==============================] - 9s 38ms/step - loss: 1.5492
Epoch 5/20
172/172 [==============================] - 9s 38ms/step - loss: 1.4501
Epoch 6/20
172/172 [==============================] - 9s 38ms/step - loss: 1.3836
Epoch 7/20
172/172 [==============================] - 9s 38ms/step - loss: 1.3309
Epoch 8/20
172/172 [==============================] - 9s 39ms/step - loss: 1.2858
Epoch 9/20
172/172 [==============================] - 9s 38ms/step - loss: 1.2445
Epoch 10/20
172/172 [==============================] - 9s 38ms/step - loss: 1.2056
Epoch 11/20
172/172 [==============================] - 9s 38ms/step - loss: 1.1655
Epoch 12/20
172/172 [==============================] - 9s 38ms/step - loss: 1.1234
Epoch 13/20


<div class="alert alert-block alert-warning">
<h2> 6. Generate text </h2> 
<div id="Text"></div>
</div>

The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.

![To generate text the model's output is fed back to the input](https://www.tensorflow.org/static/text/tutorials/images/text_generation_sampling.png)

Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.

Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.


The following makes a single step prediction:

In [38]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [39]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Run it in a loop to generate some text. Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [40]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
By my troth, or promise to a-poilthood studies.
The good wors, if thou couldst, thou sleepest tribuar.
While we met what may she break it? I have perchant
Tives our court here: be thou, if not dream of,
Let him not so fitting for Berry Pate.

BUCKINGHAM:
Exforward he, Biondello, buk it was?
And I for it work it, to the son.

GREMIO:
Why, she concetious execution?

BRUTUS:
They shame for such a life.

BALTHASAR:
Now, by Saint Paul, this news is Adon't.

TRANIO:
A little power divour! speak; set your honour's law
And told you over by God home:
But, lest shepherd, being alone; for though it is ourselves,
Were he that sets his friend will have fitted by your head:
Give ope the case he looks on him that they durst is
consevered.

EDWARD:
Where is the difference fury of the deed?
Prever shrift and not answer
To give you over my life, for if Henry's leads,
With heigh of Greey and o'erwhat late Isabour, live,
I shall so, sir: but, gentle nurse, I tell you, Signior George,
And thou might

The easiest thing you can do to improve the results is to train it for longer (try EPOCHS = 30).

You can also experiment with a different start string, try adding another RNN layer to improve the model's accuracy, or adjust the temperature parameter to generate more or less random predictions.

If you want the model to generate text faster the easiest thing you can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above.

In [41]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nPatience she will not.\n\nNORTHUMBERLAND:\nThen all the kinsman, near is, he in equasts:\nThy sovereign's enemies are part, and thou withdrawn us away.\n\nPETESTI:\nHenry of Lord of Bartogather\nWith such as other favour.\nEven as thy father, and his not atturned,\nAnd press'd some pretencier from the Valous,\nwild find his lance; to the entirtion of this\ntrust, I thank you yet in mind of his truth: but thou\nShouldst Gaunt, thy suger'a tellow Rithan!\nThe meeting of your sorrow and thy son.\n\nKING RICHARD III:\nAll puckle, as the dotion spectarly\nAgainst a tear; this shall die, even so,\nTo whom thou camest he affrights him on courage?\nI prithee, sit by the fault out with it:\nFor yinger lord pannot could behold him that which I\nproceed, come, though you in the custom of\nyet they think well, then repair to be prevented.\n\nGREMIO:\nWhat last is kind?\n\nISABELLA:\nO, I mess go-ng.\n\nCURTIS:\nThou slew'st of some track out I cannot, but knows I cannot hold 

<div class="alert alert-block alert-warning">
<h2> 7. Export the generator </h2> 
<div id="Export"></div>
</div>

This single-step model can easily be [saved and restored](https://www.tensorflow.org/guide/saved_model), allowing you to use it anywhere a `tf.saved_model` is accepted.

In [42]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [43]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
By heaven, she did lose here to go about you.

First Lady:
I have no pown and give it you
Bianca, s


<div class="alert alert-block alert-warning">
<h2> 8. Advanced: Customized Training </h2> 
<div id="Advance"></div>
</div>
The above training procedure is simple, but does not give you much control.
It uses teacher-forcing which prevents bad predictions from being fed back to the model, so the model never learns to recover from mistakes.

So now that you've seen how to run the model manually next you'll implement the training loop. This gives a starting point if, for example, you want to implement _curriculum  learning_ to help stabilize the model's open-loop output.

The most important part of a custom training loop is the train step function.

Use `tf.GradientTape` to track the gradients. You can learn more about this approach by reading the [eager execution guide](https://www.tensorflow.org/guide/eager).

The basic procedure is:

1. Execute the model and calculate the loss under a `tf.GradientTape`.
2. Calculate the updates and apply them to the model using the optimizer.

In [44]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

The above implementation of the train_step method follows Keras' train_step conventions. This is optional, but it allows you to change the behavior of the train step and still use keras' Model.compile and Model.fit methods.

In [45]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [46]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [47]:
model.fit(dataset, epochs=1)

172/172 [==============================] - 12s 41ms/step - loss: 2.7209


Or if you need more control, you can write your own complete custom training loop:

In [48]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.1953
Epoch 1 Batch 50 Loss 2.0773
Epoch 1 Batch 100 Loss 1.9382
Epoch 1 Batch 150 Loss 1.8584

Epoch 1 Loss: 1.9891
Time taken for 1 epoch 10.49 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.8125
Epoch 2 Batch 50 Loss 1.7367
Epoch 2 Batch 100 Loss 1.7160
Epoch 2 Batch 150 Loss 1.6032

Epoch 2 Loss: 1.7106
Time taken for 1 epoch 8.45 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.6418
Epoch 3 Batch 50 Loss 1.5669
Epoch 3 Batch 100 Loss 1.5260
Epoch 3 Batch 150 Loss 1.5123

Epoch 3 Loss: 1.5475
Time taken for 1 epoch 8.51 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.5006
Epoch 4 Batch 50 Loss 1.4753
Epoch 4 Batch 100 Loss 1.4581
Epoch 4 Batch 150 Loss 1.4543

Epoch 4 Loss: 1.4492
Time taken for 1 epoch 8.49 sec
________________________________________________________________________

In [49]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

start = time.time()


states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]
print(result)
i = 0
for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  if( i<3):
    print(next_char)
  i+=1
  result.append(next_char)
  if(i==4):
    print('\n\n' + '_'*80)
    print(result)
    print('\n\n' + '_'*80)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

[<tf.Tensor: shape=(5,), dtype=string, numpy=
array([b'ROMEO:', b'ROMEO:', b'ROMEO:', b'ROMEO:', b'ROMEO:'],
      dtype=object)>]
tf.Tensor([b'\n' b'\n' b'\n' b'\n' b'\n'], shape=(5,), dtype=string)
tf.Tensor([b'W' b'N' b'T' b'S' b'T'], shape=(5,), dtype=string)
tf.Tensor([b'h' b'o' b'h' b't' b'h'], shape=(5,), dtype=string)


________________________________________________________________________________
[<tf.Tensor: shape=(5,), dtype=string, numpy=
array([b'ROMEO:', b'ROMEO:', b'ROMEO:', b'ROMEO:', b'ROMEO:'],
      dtype=object)>, <tf.Tensor: shape=(5,), dtype=string, numpy=array([b'\n', b'\n', b'\n', b'\n', b'\n'], dtype=object)>, <tf.Tensor: shape=(5,), dtype=string, numpy=array([b'W', b'N', b'T', b'S', b'T'], dtype=object)>, <tf.Tensor: shape=(5,), dtype=string, numpy=array([b'h', b'o', b'h', b't', b'h'], dtype=object)>, <tf.Tensor: shape=(5,), dtype=string, numpy=array([b'y', b',', b'a', b'r', b'a'], dtype=object)>]


___________________________________________________________

<div class="alert alert-block alert-success" style="font-size:30px">
    <b>Thank you for coming to the end 🎉</b>
    <br>If it was useful to you, then I will be glad to see your vote.<br>
    </div>